In [23]:
import pandas as pd
import json
import numpy as np

In [24]:
test_file = "faa_plmarker.jsonl"
data_dir = "ace05" # ace05 or scierc
model = "albert" # bert, albert, or scibert
data_abbr = "ace05" # ace05 or sci
ner_model = f"PL-Marker/{data_abbr}ner_models/{data_abbr}ner-{model}-faa"
re_model = f"PL-Marker/{data_abbr}re_models/{data_abbr}re-{model}-faa"
results_suffix = "jun17"

In [25]:
with open(f"PL-Marker/{data_dir}/{test_file}") as f:
    test_str = f.read()

test = []
for test_doc_str in test_str.split('\n'):
    try:
        test.append(json.loads(test_doc_str))
    except:
        print(f"could not parse: {test_doc_str}")

could not parse: 


In [26]:
len(test[1]['sentences'])

3

In [27]:
idx2word = {}
for idoc in range(len(test)):
    doc = test[idoc]

    idx2word[idoc] = {}
    curr_idx = 0
    for sentence in test[idoc]['sentences']:
        for word in sentence:
            if word not in idx2word[idoc]:
                idx2word[idoc][curr_idx] = word
                curr_idx = curr_idx + 1

In [28]:
with open(f"{ner_model}/ent_pred_test.json") as f:
    ent_pred_str = f.read()

ent_pred = []
for ent_pred_doc_str in ent_pred_str.split('\n'):
    try:
        ent_pred.append(json.loads(ent_pred_doc_str))
    except:
        print(f"could not parse: {ent_pred_doc_str}")

could not parse: 


In [29]:
ent_pred[0].keys()

dict_keys(['sentences', 'ner', 'relations', 'doc_key', 'predicted_ner'])

In [30]:
len(ent_pred[1]['sentences'])

3

In [31]:
def get_span(idoc, sentence, idx0, idx1):
    start_span = sentence.index(idx2word[idoc][idx0])
    end_span = sentence.index(idx2word[idoc][idx1], start_span)
    while True:
        try:
            start_span = sentence.index(idx2word[idoc][idx0], start_span+1, end_span)
        except:
            break
    return sentence[start_span:end_span+1]

In [32]:
for idoc in range(len(ent_pred)):
    print(f"*** Document #{idoc} ****\n")
    for isentence in range(len(ent_pred[idoc]["sentences"])):
        print(' '.join(ent_pred[idoc]['sentences'][isentence]))
        print("\nPredicted Named Entities:")
        for entity in ent_pred[idoc]['predicted_ner'][isentence]:
            print(entity)
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            print(f"{span} : {entity[2]}")
        print("\n----------------\n")
    print("********************")

*** Document #0 ****

tailwheel cocked right prior to tkof .

Predicted Named Entities:

----------------

********************
*** Document #1 ****

tow plane became airborne then settled .

Predicted Named Entities:
[1, 1, 'VEH']
plane : VEH

----------------

student thought tow in trouble & released .

Predicted Named Entities:
[7, 7, 'PER']
student : PER

----------------

hit tree .

Predicted Named Entities:

----------------

********************
*** Document #2 ****

2nd ils apch , acft 's g/s inop .

Predicted Named Entities:

----------------

lom tuned to wrong freq .

Predicted Named Entities:

----------------

********************
*** Document #3 ****

plt noted soft r brake pedal drg taxi to tkof .

Predicted Named Entities:

----------------

flt rtnd springfield due soft brake strong winds bos

Predicted Named Entities:

----------------

********************
*** Document #4 ****

taxi off hard sfc due tfc r main gr broke through roof of a washed out underground tunne

In [33]:
results_dict = {"c5_id":[], "doc_idx":[], "sent_idx":[],  "c119_input":[], "sentence":[], "entities":[], "labels":[]}
og_data = pd.read_csv("../../OMIn_dataset/data/FAA_data/Maintenance_Text_data_nona.csv")

for idoc in range(len(ent_pred)):
    for isentence, sentence in enumerate(ent_pred[idoc]["sentences"]):
        sentence = ent_pred[idoc]['sentences'][isentence]
        og_sentence = og_data["c119"].iat[idoc]
        c5_id = og_data["c5"].iat[idoc]
        
        for entity in ent_pred[idoc]['predicted_ner'][isentence]:
            results_dict["doc_idx"].append(idoc)
            results_dict["sent_idx"].append(isentence)
            results_dict["c5_id"].append(c5_id)
            results_dict["c119_input"].append(og_sentence)
            results_dict["sentence"].append(' '.join(sentence))
            
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            label = entity[2]
            results_dict["entities"].append(span)
            results_dict["labels"].append(label)

In [34]:
pd.DataFrame(results_dict).to_csv(f"../../OMIn_dataset/data/results/pl-marker/pl-marker_{data_dir}_{model}_NER_{results_suffix}.csv", index=False)

In [35]:
with open(f"{re_model}/pred_results.json") as f:
    pred_results_str = f.readlines()[0]
    pred_results = json.loads(pred_results_str)

In [36]:
pred_results.keys()

dict_keys(['1', '25', '41', '46', '47', '50', '52', '57', '59', '61', '64', '65', '66', '67', '71', '72', '74', '78', '79', '80', '84', '88', '91', '97', '100', '101', '102', '107', '117', '118', '121', '127', '129', '130', '131', '132', '133', '134', '139', '140', '142', '145', '146', '149', '157', '159', '160', '166', '168', '171', '179', '184', '187', '189', '191', '202', '203', '204', '205', '209', '210', '211', '213', '217', '219', '220', '221', '230', '235', '237', '238', '242', '244', '245', '246', '250', '251', '252', '254', '255', '256', '258', '260', '261', '262', '264', '265', '266', '268', '269', '274', '276', '277', '278', '279', '282', '284', '285', '286', '288', '289', '293', '295', '298', '300', '307', '308', '310', '311', '314', '315', '317', '319', '320', '322', '323', '326', '327', '329', '331', '334', '335', '336', '337', '338', '343', '348', '349', '350', '352', '355', '356', '357', '358', '359', '362', '363', '368', '370', '372', '373', '374', '377', '380', '381',

In [37]:
for doc_str in pred_results.keys():
    idoc = int(doc_str)
    print(f"*** Document #{idoc} ****")
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        print(' '.join(sentence))
        if len(triples) > 0:
            for triple in triples:
                print(f"\t{' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1]))} --> {triple[2]} --> {' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1]))}")
        else:
            print("\tNo triples found")
        print("\n--------------\n")

*** Document #1 ****
tow plane became airborne then settled .
	No triples found

--------------

student thought tow in trouble & released .
	No triples found

--------------

*** Document #25 ****
plt continued taxiing after total hyd failure .
	No triples found

--------------

*** Document #41 ****
pilot failed to check position of gear handle .
	No triples found

--------------

*** Document #46 ****
new aircraft owner not aware of purpose or operation of quick drain .
	No triples found

--------------

*** Document #47 ****
pilot failed to remove tow bar before takeoff .
	No triples found

--------------

*** Document #50 ****
acft dispatcher harrassment of pilot .
	No triples found

--------------

pilot forgot to remove tiedrope .
	No triples found

--------------

*** Document #52 ****
no pilot certificate , no annual .
	No triples found

--------------

*** Document #57 ****
pilot intended to fly locally after recent blizzard .
	No triples found

--------------

*** Document #

In [38]:
results_dict = {"c5_id":[], "doc_idx":[], "sent_idx":[], "c119_input":[], "sentence":[], "head":[],"relation":[], "tail":[]}
og_data = pd.read_csv("../../OMIn_dataset/data/FAA_data/Maintenance_Text_data_nona.csv")

for doc_str in pred_results.keys():
    idoc = int(doc_str)
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        og_sentence = og_data["c119"].iat[idoc]
        c5_id = og_data["c5"].iat[idoc]

        for triple in triples:
            results_dict["doc_idx"].append(idoc)
            results_dict["sent_idx"].append(isentence)
            results_dict["c5_id"].append(c5_id)
            results_dict["c119_input"].append(og_sentence)
            results_dict["sentence"].append(' '.join(sentence))
            results_dict["head"].append(' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1])))
            results_dict["relation"].append(triple[2])
            results_dict["tail"].append(' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1])))

In [39]:
for i_triple in range(len(results_dict["doc_idx"])):
    print(f"{results_dict['doc_idx'][i_triple]}, {results_dict['c119_input'][i_triple]}\t---from---\t{og_data['c119'].iat[results_dict['doc_idx'][i_triple]]}")

149, WHILE TAXIING TO MAINTENANCE HANGAR, PILOT LOST NOSE WHEEL STEERING. WINGTIP HIT HANGAR, CAUGHT ON FIRE.           	---from---	WHILE TAXIING TO MAINTENANCE HANGAR, PILOT LOST NOSE WHEEL STEERING. WINGTIP HIT HANGAR, CAUGHT ON FIRE.           
209, FORCED LANDING DUE TO ENGINE FAILURE. THE PILOT SAID WATER FOUND IN FUEL AND IN COMPANY OWNED FUEL TANKER.         	---from---	FORCED LANDING DUE TO ENGINE FAILURE. THE PILOT SAID WATER FOUND IN FUEL AND IN COMPANY OWNED FUEL TANKER.         
211, INSTRUCTOR CHECKING OUT PILOT IN MULTIENGINE AIRPLANE. STUDENT FEATHERED WRONG ENGINE. EMERGENCY LANDING.          	---from---	INSTRUCTOR CHECKING OUT PILOT IN MULTIENGINE AIRPLANE. STUDENT FEATHERED WRONG ENGINE. EMERGENCY LANDING.          
237, PILOT FLEW AIRCRAFT NOT SIGNED OFF AFTER INSPECTION. LOST UPPER ENGINE COWLING IN FLIGHT. HAD NOT BEEN FASTENED.   	---from---	PILOT FLEW AIRCRAFT NOT SIGNED OFF AFTER INSPECTION. LOST UPPER ENGINE COWLING IN FLIGHT. HAD NOT BEEN FASTENED.   
268, ENG

In [40]:
pd.DataFrame(results_dict).to_csv(f"../../OMIn_dataset/data/results/pl-marker/pl-marker_{data_dir}_{model}_RE_{results_suffix}.csv")